In [1]:
import warnings
warnings.simplefilter(action='ignore')

In [2]:
import scanpy as sc
import torch
import scarches as sca
import numpy as np

 captum (see https://github.com/pytorch/captum).


In [3]:
sc.set_figure_params(frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)

In [5]:
adata = sc.read('../dataset/tyser.h5ad')
hips = sc.read("../dataset/hiPSC.h5ad")

In [7]:
sca.utils.add_annotations(adata, '../metadata/reactome.gmt', min_genes=12, clean=True)

In [8]:
adata._inplace_subset_var(adata.varm['I'].sum(1)>0)

In [9]:
sc.pp.normalize_total(adata)

In [10]:
sc.pp.log1p(adata)

In [11]:
adata.obs['batch'] = 'reference'

In [12]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    batch_key="batch",
    subset=True)

In [13]:
select_terms = adata.varm['I'].sum(0) > 12
adata.uns['terms'] = np.array(adata.uns['terms'])[select_terms].tolist()
adata.varm['I'] = adata.varm['I'][:, select_terms]

In [14]:
adata._inplace_subset_var(adata.varm['I'].sum(1) > 0)

In [15]:
adata.X = adata.X.copy()

In [16]:
missing_genes = [gene for gene in adata.var_names if gene not in hips.var_names]
print(f"Aantal genen in hips die niet in tyser staan: {len(missing_genes)}")
print("Ontbrekende genen:", missing_genes)

Aantal genen in hips die niet in tyser staan: 136
Ontbrekende genen: ['ABCA10', 'ACMSD', 'ACSL5', 'AGXT2', 'ALB', 'APOC3', 'BBOX1', 'BLNK', 'BMX', 'C1QA', 'C1QB', 'C1QC', 'C3AR1', 'C6', 'C8A', 'C8B', 'C9', 'CACNG3', 'CCL22', 'CCL3L3', 'CCR3', 'CCR4', 'CD180', 'CD40LG', 'CD80', 'CETP', 'CGB5', 'CGB8', 'CHRM5', 'CHRNB3', 'CLDN22', 'CTGF', 'CX3CR1', 'CYBB', 'CYP2W1', 'CYP4B1', 'CYP4F2', 'DBH', 'DEFB104A', 'ENPP7', 'F2RL3', 'FCGR1A', 'FCGR2B', 'FMO3', 'FPR1', 'GABRA6', 'GATA1', 'GBA3', 'GBP5', 'GLRA2', 'GLYAT', 'GP9', 'GPR65', 'GPX1', 'HAL', 'HBB', 'HBD', 'HBE1', 'HCAR3', 'HK3', 'HMGCS2', 'HRG', 'HTR5A', 'IL1RN', 'IL2RG', 'IL3', 'ITGAL', 'KCNJ16', 'KCNV2', 'LILRA1', 'LILRB2', 'LIPC', 'LY86', 'LYVE1', 'MBOAT4', 'MC3R', 'MMP13', 'MMRN1', 'MPL', 'MUC6', 'MYBPC1', 'NLRC4', 'NMUR2', 'NPPA', 'NPSR1', 'NR0B2', 'NTSR2', 'OAS2', 'OLR1', 'OR2A42', 'OR2L13', 'OR2W3', 'OR3A1', 'OR51E1', 'OR51E2', 'OR51L1', 'OR6V1', 'OR8K1', 'PIK3CG', 'PIK3R6', 'PLA2G16', 'PLA2G4D', 'PLEK', 'PLG', 'PLXNB3', 'PNLIPRP2',

In [20]:
commom = adata.var_names.intersection(hips.var_names)
hips = hips[:, commom].copy()
adata = adata[:, commom].copy()


In [21]:
adata

AnnData object with n_obs × n_vars = 1069 × 1841
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Source.Name', 'Characteristics.sampling.site.', 'Characteristics.inferred.cell.type...authors.labels.', 'Characteristics.inferred.cell.type...ontology.labels.', 'cluster_id', 'sub_cluster', 'origin', 'run', 'sample_name', 'timepoint', 'percent.mt', 'batch'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'terms', 'log1p', 'hvg'
    varm: 'I'

In [ ]:
intr_cvae = sca.models.EXPIMAP(
    adata=adata,
    condition_key='batch',
    hidden_layer_sizes=[256, 256, 256],
    recon_loss='nb'
)

In [ ]:
ALPHA = 0.7

adata.X = adata.X.astype(np.float32)

In [ ]:
early_stopping_kwargs = {
    "early_stopping_metric": "val_unweighted_loss", # val_unweighted_loss
    "threshold": 0,
    "patience": 50,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}
intr_cvae.train(
    n_epochs=400,
    alpha_epoch_anneal=100,
    alpha=ALPHA,
    alpha_kl=0.5,
    weight_decay=0.,
    early_stopping_kwargs=early_stopping_kwargs,
    use_early_stopping=True,
    monitor_only_val=False,
    seed=2020,
)

In [ ]:
adata.obsm['X_cvae'] = intr_cvae.get_latent(mean=MEAN, only_active=True)

In [ ]:
sc.pp.neighbors(adata, use_rep='X_cvae')

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['Characteristics.inferred.cell.type...authors.labels.'], frameon=False)

In [ ]:
hips.obs['batch'] = 'query'

In [ ]:
hips.X = hips.X.astype(np.float32)


In [ ]:
q_intr_cvae = sca.models.EXPIMAP.load_query_data(hips, intr_cvae)

In [ ]:
q_intr_cvae.train(n_epochs=400, alpha_epoch_anneal=100, weight_decay=0., alpha_kl=0.1, seed=2020, use_early_stopping=True)


In [ ]:
dataset_samen = sc.AnnData.concatenate(adata, hips, batch_key='batch_join', uns_merge='same')
dataset_samen


In [ ]:
dataset_samen.X = dataset_samen.X.astype(np.float32)


In [ ]:
dataset_samen.obsm['X_cvae'] = q_intr_cvae.get_latent(dataset_samen.X, dataset_samen.obs['batch'], mean=MEAN, only_active=True)

In [ ]:
sc.pp.neighbors(dataset_samen, use_rep='X_cvae')

In [ ]:
sc.tl.umap(dataset_samen)


In [ ]:
sc.pl.umap(dataset_samen, color=['batch'], frameon=False, wspace=0.6)


In [ ]:
sc.pl.umap(dataset_samen, color=['Characteristics.inferred.cell.type...authors.labels.'], frameon=False, wspace=0.6)

In [ ]:

sc.pl.umap(dataset_samen, color=['seurat_clusters'], frameon=False, wspace=0.6)

In [ ]:
dataset_samen.obs['orig.ident'] = dataset_samen.obs['orig.ident'].astype(str)


dataset_samen.write('hipstyser_integrated.h5ad')